In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import datasets
import torch
import shap
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import f1_score,  recall_score, precision_score, multilabel_confusion_matrix

In [2]:
tsd_train = pd.read_csv('tsd_test.csv')
tsd_train.dtypes

spans    object
text     object
dtype: object

In [3]:
tsd_train['label'] = tsd_train.apply(lambda row: 1 if len(row['spans']) > 2 else 0, axis = 1)
tsd_train['toxic']= tsd_train['label']
tsd_train['non-toxic']= 1-tsd_train['label']
tsd_train['labels'] = tsd_train.loc[:, ['toxic','non-toxic']].values.tolist()
tsd_train = tsd_train.drop(['label'], axis=1)
tsd_train.head(1)

,spans,text,toxic,non-toxic,labels
0,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...",That's right. They are not normal. And I am st...,1,0,"[1, 0]"


In [4]:
tsd_train_dataset = datasets.Dataset.from_pandas(tsd_train)
tsd_train_dataset[3]

{'spans': '[]',
 'text': 'The parallels between the ANC and the Sicilian Mafia are glaring. The ANC has always been run by a few "families" who treat the state as \'turf\' ; as just one big piggy bank for their self-enrichment. The government basically believes we all just work for them. They aren\'t a democratic government at all, but use the appearances of democracy to give their entitlement practices a mask of legitimacy. The poor and ignorant have been fooled for so long, but people are slowly coming to the ANC for what it is: a self-serving Mafia!',
 'toxic': 0,
 'non-toxic': 1,
 'labels': [0, 1]}

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
tsd_train_dataset = tsd_train_dataset.map(
                lambda x: tokenizer(
                        x['text'], max_length=256, padding='max_length',
                        truncation=True
                    ), batched=True
)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [7]:
tsd_train_dataset

Dataset({
    features: ['spans', 'text', 'toxic', 'non-toxic', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

In [8]:
tsd_train_dataset = tsd_train_dataset.remove_columns(['text', 'spans', 'toxic', 'non-toxic','labels'])
tsd_train_dataset.set_format(type='torch', output_all_columns=True)

In [9]:
tsd_train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

In [10]:
batch_size = 16
loader = torch.utils.data.DataLoader(tsd_train_dataset,batch_size=batch_size)
PATH = './model_save3'
model = BertForSequenceClassification.from_pretrained(PATH,output_hidden_states = True,)

In [11]:
sigmoid = torch.nn.Sigmoid()

eval_preds, eval_labels = [], []
eval_accuracy = 0

input_ids = tsd_train_dataset['input_ids']
attention_mask = tsd_train_dataset['attention_mask']


with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    probability = outputs.logits.sigmoid()

In [12]:
sequence_pred = np.array(probability)
sequence_pred [sequence_pred  >= 0.5] = 1
sequence_pred [sequence_pred  < 0.5] = 0
seq_pred_list = sequence_pred.tolist()
tsd_train['seq_pred'] = seq_pred_list

In [13]:
#tsd_train.remove_columns(['spans', 'toxic', 'non-toxic','labels'])
tsd_train.head(1)

,spans,text,toxic,non-toxic,labels,seq_pred
0,"[84, 85, 86, 87, 88, 89, 90, 91, 133, 134, 135...",That's right. They are not normal. And I am st...,1,0,"[1, 0]","[1.0, 0.0]"


In [14]:
list_text = []
list_text = tsd_train['text']

In [15]:
from transformers import AutoTokenizer
tokenizer2 = AutoTokenizer.from_pretrained('bert-base-uncased')

list_tokens = []
list_starts=[]
list_ends = []
list_sent_idx = []
for sentence_idx , text in enumerate(tsd_train['text']):
    encoding_test= tokenizer2(text, max_length=256, padding='max_length',truncation=True)
    tokens = []
    starts=[]
    ends = []
    for idx ,token in enumerate(encoding_test.tokens()):
        if ((token =='[CLS]') or (token =='[SEP]') or (token =='[PAD]')):
            start = 9999999999
            end = 9999999999
        else:
            start,end = encoding_test.token_to_chars(idx)
        tokens.append(token)
        starts.append(start)
        ends.append(end)
    list_sent_idx.append(sentence_idx)
    list_tokens.append(tokens)
    list_starts.append(starts)
    list_ends.append(ends)

all_sent_indx = []
all_tokens =[]
all_starts = []
all_ends = []
for sent_idx in list_sent_idx:
    for token_idx, token in enumerate(list_tokens[sent_idx]):
        all_sent_indx.append(sent_idx)
        all_tokens.append(token)
        all_starts.append(list_starts[sent_idx][token_idx])
        all_ends.append(list_ends[sent_idx][token_idx])

all_index = {'sentence_idx':all_sent_indx,'token':all_tokens,'start':all_starts,'end':all_ends}
idx_df = pd.DataFrame(all_index)
idx_df = idx_df.drop(idx_df[idx_df.start == 9999999999].index)

In [16]:
idx_df.head(1)

,sentence_idx,token,start,end
1,0,that,0,4


In [17]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=256, truncation=True) for v in x]).to(device)
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

In [19]:
explainer = shap.Explainer(f, tokenizer)
shap_values = explainer(list_text, fixed_context=1)

Partition explainer: 2001it [21:31,  1.54it/s]                                  


In [20]:
len(shap_values)

2000

In [21]:
select_shap_values=[]
for i in range(len(shap_values)):
    select_shap_values.append(shap_values[i].values[1:-1])

select_shap_values = [l.tolist() for l in select_shap_values]

In [22]:
for i in range(len(pd.unique(idx_df['sentence_idx']))):
    shap_len = len(select_shap_values[i])
    if shap_len > 254:
        select_shap_values[i] = select_shap_values[i][0:254]

In [23]:
pred_list =[]
for x in range(len(shap_values)):
    pred_list.append([seq_pred_list[x] for i in range(len(select_shap_values[x]))])

In [24]:
from itertools import chain
all_predictions = list(chain(*pred_list))

In [25]:
all_shap_base_values = []
for x in range(len(shap_values)):
    all_shap_base_values.extend([shap_values[x].base_values for i in range(len(select_shap_values[x]))])

In [26]:
len(all_shap_base_values)

86201

In [27]:
len(idx_df)

86201

In [28]:
len(all_predictions)

86201

In [29]:
from itertools import chain
all_shap_values = list(chain(*select_shap_values))
#all_shap_values = [item for sublist in select_shap_values for item in sublist]   

In [30]:
len(all_shap_values)

86201

In [31]:
idx_df['shap_value'] = all_shap_values
idx_df['shap_base_value'] = all_shap_base_values
idx_df['sentiment'] = all_predictions

In [32]:
# idx_df = idx_df.drop(columns=['token'])
# idx_df.to_hdf('classifier_testing_data_0105_01.h5', key='idx_df', mode='w')

In [42]:
idx_df[idx_df.sentence_idx == 0]

,sentence_idx,token,start,end,shap_value,shap_base_value,sentiment
1,0,that,0,4,-0.291169,8.962646,"[1.0, 0.0]"
2,0,',4,5,0.027174,8.962646,"[1.0, 0.0]"
3,0,s,5,6,-0.018401,8.962646,"[1.0, 0.0]"
4,0,right,7,12,0.179220,8.962646,"[1.0, 0.0]"
5,0,.,12,13,0.107237,8.962646,"[1.0, 0.0]"
6,0,they,14,18,-0.230506,8.962646,"[1.0, 0.0]"
7,0,are,19,22,-0.259135,8.962646,"[1.0, 0.0]"
8,0,not,23,26,-0.058490,8.962646,"[1.0, 0.0]"
9,0,normal,27,33,0.196961,8.962646,"[1.0, 0.0]"
10,0,.,33,34,0.250125,8.962646,"[1.0, 0.0]"


In [39]:
idx_df.to_hdf('classifier_testing_data_0105_02.h5', key='idx_df', mode='w')


your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['token', 'sentiment'], dtype='object')]

